In [1]:
%load_ext pyprql.magic
%prql duckdb://

There's a new jupysql version available (0.10.0), you're running 0.9.0. To upgrade: pip install jupysql --upgrade


In [2]:
import duckdb
import prql_python as prql

In [3]:
# Path
path = "../data/"
data_path = path + "datathon/"

In [4]:
prql_query = """
    from vgsales_long
    filter Region != "Global_Sales"
"""

sql_statement = prql.compile(prql_query)

In [5]:
# Source
vgsales_long = duckdb.read_parquet(data_path + "vgsales.parquet")
vgsales = duckdb.sql(sql_statement)

In [6]:
# Source
pricing = duckdb.read_parquet(data_path + "pricing.parquet")
department = duckdb.read_parquet(data_path + "department.parquet")
metrics = duckdb.read_parquet(data_path + "metrics.parquet")

In [7]:
%prql from pricing | take 3

,size,vcpu,memory,cost_per_hour
0,t3a.medium,2,4.0,0.0376
1,t3.large,2,8.0,0.0832
2,t3.xlarge,4,16.0,0.1664


### 1.2 How many departments use the appliances of the Data Platform?

In [8]:
prql_query = """
    from metrics
    join side:left department (==id)
"""
options = prql.CompileOptions(format=False)
sql_statement = prql.compile(prql_query, options)
metrics_department = duckdb.sql(sql_statement)

In [9]:
prql_query = """
    from metrics_department
    group {id, department, disk_size, `type`, size} (
        aggregate {}
    )
"""

sql_statement = prql.compile(prql_query)
metrics_unique = duckdb.sql(sql_statement)

In [10]:
%%prql from metrics_unique
group department (
    aggregate {count_department = count(department)}
)
sort {-count_department, -department}


,department,count_department
0,Sales,10
1,IT,6
2,HR,6
3,Engineering,5
4,Operations,3
5,Marketing,3


### 1.3 What is the most popular appliance size used by all departments? And how many of those popular sizes did you find in the whole dataset?

In [11]:
%%prql from metrics_unique
group size (
    aggregate {count_size = count(size)}
)
sort {-count_size, -size}

,size,count_size
0,t3a.medium,21
1,t3a.2xlarge,2
2,r5a.2xlarge,2
3,c5.4xlarge,2
4,t3a.xlarge,1
5,t3.xlarge,1
6,t3.large,1
7,r5.2xlarge,1
8,m5d.xlarge,1
9,g3s.xlarge,1


### 2.1 Which is the most popular appliance type per department?

In [12]:
%%prql from metrics_unique
group {department, `type`} (
    aggregate {type_per_department = count(`type`)}
)
sort {department, -type_per_department}

,department,type,type_per_department
0,Engineering,jupyter,3
1,Engineering,knime,1
2,Engineering,rstudio,1
3,HR,knime,5
4,HR,deeplearning,1
5,IT,jupyter,3
6,IT,rstudio,2
7,IT,knime,1
8,Marketing,matlab,2
9,Marketing,deeplearning,1


### 2.2 Wich appliance size had the lowest vCPU utilization over the full time range of the dataset based on the listed metrics? Calculate a value with 6 digits after zero for each metric:

In [13]:
prql_query = """
    from metrics
    group {id, data_timestamp, size, vcpu} (
        aggregate {}
    )
"""

sql_statement = prql.compile(prql_query)
vcpu_by_size = duckdb.sql(sql_statement)

In [14]:
%%prql
from vcpu_by_size
group size (
    aggregate {
        minimum = min(vcpu),
        mean = average(vcpu)
    }
)
sort {minimum}

,size,minimum,mean
0,r5a.2xlarge,0.000165,0.545604
1,t3a.medium,0.010630,11.027830
2,c5.4xlarge,0.105102,6.329230
3,t3a.2xlarge,0.212592,0.537950
4,t3.xlarge,0.256002,2.191348
5,r5.2xlarge,0.302333,1.507907
6,t3a.xlarge,0.368000,1.903183
7,g3s.xlarge,0.391572,0.450599
8,m5d.xlarge,0.641000,15.238145
9,t3.large,0.947409,1.075165


### 2.3 Which department has used the most appliances between 15.12.2022 and 16.01.2023?  How many appliances did they use in this time range?

In [15]:
%%prql
from metrics_department
filter data_timestamp > @2022-12-15
filter data_timestamp < @2023-01-16
group department (
    aggregate{appliance_count = count_distinct(id)}
)
sort {-appliance_count}

,department,appliance_count
0,Sales,6
1,Engineering,4
2,IT,3
3,HR,3
4,Marketing,1
5,Operations,1


### 2.4 What is the most expensive size of an appliance used in the Data Platform in terms of hours used per department?

In [16]:
prql_query = """
    from m=metrics
    join side:left d=department (==id)
    join side:left p=pricing (m.size == p.size)
    group {d.department, m.size, p.cost_per_hour, m.data_timestamp}(
        aggregate{}
    )
"""

sql_statement = prql.compile(prql_query)
metrics_extended = duckdb.sql(sql_statement)

In [17]:
prql_query = """
    from metrics_extended
    group {department, size, cost_per_hour}(
        aggregate{timestamp_count = count(data_timestamp)}
    )
"""

sql_statement = prql.compile(prql_query)
data_usage = duckdb.sql(sql_statement)

In [18]:
%%prql
from data_usage
derive cost = timestamp_count / 12 * cost_per_hour
sort {-department, -cost}

,department,size,cost_per_hour,timestamp_count,cost
0,Sales,g3s.xlarge,0.7500,15864,991.500000
1,Sales,r5.2xlarge,0.5040,16704,701.568000
2,Sales,m5d.xlarge,0.2260,14128,266.077333
3,Sales,t3a.medium,0.0376,3423,10.725400
4,Operations,t3a.2xlarge,0.3008,916,22.961067
5,Operations,t3a.medium,0.0376,47,0.147267
6,Marketing,t3.xlarge,0.1664,3282,45.510400
7,Marketing,t3.large,0.0832,1104,7.654400
8,Marketing,t3a.medium,0.0376,13,0.040733
9,IT,r5a.2xlarge,0.4520,8737,329.093667


### 3.1 Which fields are important to find out if an appliance is idle - meaning that an appliance is running but no action is performed on it?  Sort the correct values in alphabetic order, before submitting your response.

### 3.2 Which appliances were idle and when?

In [19]:
%%prql
from metrics
select {size, net_in, net_out}
group size (aggregate{
        max_net_in = max(net_in),
        max_net_out = max(net_out)
})

,size,max_net_in,max_net_out
0,t3a.2xlarge,5.866000e+07,1.073484e+08
1,t3a.medium,3.003785e+08,1.634845e+08
2,c5.4xlarge,9.233983e+07,2.798665e+08
3,r5a.2xlarge,1.960112e+09,1.182566e+09
4,t3a.xlarge,1.212759e+08,1.889854e+07
5,t3.xlarge,1.683825e+08,1.801788e+08
6,r5.2xlarge,3.327275e+09,2.951368e+09
7,t3.large,1.558500e+09,6.031056e+06
8,g3s.xlarge,1.812509e+07,1.978456e+05
9,m5d.xlarge,2.796467e+09,2.759850e+09


In [20]:
prql_query = """
    from metrics
    select {size, net_in, net_out}
    group size (aggregate{
            max_net_in = max(net_in),
            max_net_out = max(net_out)
    })
"""

sql_statement = prql.compile(prql_query)
max_net_in_out = duckdb.sql(sql_statement)

In [21]:
prql_query = """
    from max_net_in_out
    derive max_network = max_net_in + max_net_out
    select {size, max_network}
"""

sql_statement = prql.compile(prql_query)
max_network = duckdb.sql(sql_statement)

In [22]:
%%prql
from m=metrics
join side:left max_network (==size)
derive {
    net_usage = m.net_in + m.net_out,
    check_network = case {
                    net_usage < max_network.max_network * 0.02 => "idle",
                    true => "running"
                    },
    check_vcpu =    case {
                    m.`type` == 'deeplearning' && m.vcpu < 10 || m.vcpu < 5 => "idle",
                    true => "running"
                    }
}
select {m.id, m.data_timestamp, check_network, check_vcpu}

,id,data_timestamp,check_network,check_vcpu
0,i-0a16592fdb1239d51,2023-01-23 11:55:00,idle,idle
1,i-0a16592fdb1239d51,2023-01-23 11:55:00,idle,idle
2,i-0a16592fdb1239d51,2023-01-23 11:55:00,idle,idle
3,i-0a16592fdb1239d51,2023-01-23 11:50:00,idle,idle
4,i-0a16592fdb1239d51,2023-01-23 11:50:00,idle,idle
...,...,...,...,...
184300,i-0ceaefba34c469fb2,2022-12-02 20:25:00,running,running
184301,i-0ceaefba34c469fb2,2022-12-02 20:20:00,idle,idle
184302,i-0ceaefba34c469fb2,2022-12-02 20:15:00,idle,idle
184303,i-0ceaefba34c469fb2,2022-12-02 20:10:00,idle,idle


In [23]:
prql_query = """
    from m=metrics
    join side:left max_network (==size)
    derive {
        net_usage = m.net_in + m.net_out,
        check_network = case {
                        net_usage < max_network.max_network * 0.02 => "idle",
                        true => "running"
                        },
        check_vcpu =    case {
                        m.`type` == 'deeplearning' && m.vcpu < 10 || m.vcpu < 5 => "idle",
                        true => "running"
                        }
    }
    select {m.id, m.data_timestamp, check_network, check_vcpu}
"""

sql_statement = prql.compile(prql_query)
check_idle = duckdb.sql(sql_statement)

In [24]:
%%prql
from check_idle
filter check_network == "running" || check_vcpu =="running"
# sort {data_timestamp, id}


,id,data_timestamp,check_network,check_vcpu
0,i-0a16592fdb1239d51,2023-01-23 11:40:00,running,idle
1,i-0a16592fdb1239d51,2023-01-23 11:40:00,running,idle
2,i-0a16592fdb1239d51,2023-01-23 11:40:00,running,idle
3,i-0a16592fdb1239d51,2023-01-23 11:35:00,running,idle
4,i-0a16592fdb1239d51,2023-01-23 11:35:00,running,idle
...,...,...,...,...
20680,i-0ceaefba34c469fb2,2022-12-02 22:15:00,idle,running
20681,i-0ceaefba34c469fb2,2022-12-02 21:40:00,idle,running
20682,i-0ceaefba34c469fb2,2022-12-02 21:35:00,idle,running
20683,i-0ceaefba34c469fb2,2022-12-02 20:35:00,idle,running
